This notebook is a first attempt to translate the FORTRAN90 code from MTV2016 into python
and to replicate the main results of this paper.

In [1]:
#Paramters for the piece-rate version

delta = 0.000226
lambe = 0.7048178
k = 22.9931
sigma_z = 0.0
rhorho1 = 4.296953
rho2 = 0.065518
zlearningprob = 0.26398515
b = 2.3105359
theta9 = 0.0
theta10 = 0.0
kweib = 11.23955
lweib = 3.46394
zshockprob = 0.0093854   


#    IF (npar>=10) THEN
#    rho3=theta(9)  
#    rho4=theta(10)  
#    ELSE IF (npar==8) THEN
#    rho3=0.0_8
#    rho4=0.0_8
#    ELSE 

In [ ]:
# Match shock z.
# First, populating the match productivity shock vector, if zpts_first is >1

zpts = zpts_first

# zvector are the actual values of the match quality, dim(zpts)
# ztransmatrix is the transition matrix, dim(zpts, zpts)

In [ ]:
# LOGNORMAL distr of z shocks

mu_temp = -0.5 * sigma_z**2 # mean of the lognormal distr, s.t. mean of the distr is 1
binsize = 1 / (zpts - 1) # binsize
xstep = 1 / resolution # keep stepping, Resolution should be much higher than no of bins (10000)

xvalue = xstep # move along the interval of z's, starting close to 1
tempreal = 0.8 * xstep * lnorm(xvalue, mu_step, sigma_z) # this value is used to weight the z-values
# properly within the bin
bincounter = 1 # start with the first bin

# zprobcdf contains the probabilities of the idiosyncratic shock, dim zpts

while bincounter < (zpts - 1):
    if lnorm(xvalue, mu_step, sigma_z) >= (bincounter * binsize): # if the bin is full..
        zprobcdf(bincounter) = lnorm(xvalue, mu_step, sigma_z) # add to the cdf
        if bincounter == 1:
            zprobcdf(bincounter) = zprobcdf(bincounter)
            zvector(bincounter) = tempreal / zprobcdf(bincounter)
        else if bincounter > 1:
            zprobcdf(bincounter) = zprobcdf(bincounter) - zprobcdf(bincounter - 1)
            zvector(bincounter) = tempreal / zprobcdf(bincounter)
        
        tempreal = 0 # reset the average counter 
        bincounter += 1
        
        while lnorm(xvalue, mu_step, sigma_z) >= (bincounter * binsize):
            zvector(bincounter) = xstep
            zprobcdf(bincounter) = lnorm(xvalue, mu_step, sigma_z)
            zprobcdf(bincounter) = 0
            bincounter += 1
            
    else:
        xvalue = xvalue + xstep
        tempreal = tempreal + (xvalue - 0.5 * xstep) * (lnorm(xvalue, mu_step, sigma_z) - lnorm(xvalue - xstep, mu_step, sigma_z))
        
# last gridpoint

tempreal = 0
xstep = xvalue / (zpts * 5)

counter1 = 1
while counter1 < 11:
    xvalue = xvalue + xstep
    tempreal = tempreal + (xvalue - 0.5 * xstep) * (lnorm(xvalue, mu_step, sigma_z) - lnorm(xvalue - xstep, mu_step, sigma_z))
    counter1 += 1
    
zvector(zpts - 1) = tempreal / ((lnorm(xvalue, mu_step, sigma_z) - zprobcdf(zpts - 2))
zprobcdf(zpts - 1) = 1
zprobcdf(zpts - 1) = 1 - zprobcdf(zpts - 2)

In [ ]:
# UNIFORM distr of z shocks

tempreal = 1 / (zpts - 1)
zprobcdf(2:zpts) = tempreal

tempreal2 = 1 / (zpts - 2)
zvector = 0.5
zprobcdf(2) = zprobcdf(2)

for zcnt in range(3, zpts+1):
    zvector(zctn) = zvector(zctn - 1) + tempreal2
    zprobcdf(zctn) = zprobcdf(zctn - 1) + tempreal
    zctn += 1